In [ ]:
import asyncio
import aiohttp
import aiodns
import pandas as pd
import socket
import os
from pathlib import Path
from urllib.parse import urlparse
from tqdm.asyncio import tqdm_asyncio
from tqdm import tqdm
import csv
import ssl
import nest_asyncio

nest_asyncio.apply()

# CONFIG
FOLDER = Path("C:\\Users\\HP\\Desktop\\PhishingDetection\\models\\model_training\\PhishingUrlData")
EXCEL_GLOB = "*.xls*"
SHEET_NAME_OUT = "Active_Domains"
POSSIBLE_COLUMN_NAMES = [
    "Identified Phishing/Suspected Domain Name",
    "Identified Phishing/Suspected Domain",
    "Domain",
    "URL",
    "Identified Phishing/Suspected Domain Name ",
    "Identified Phishing/Suspected Domain Name  ",
    "Phishing/Suspected Domain Name",
    "Identified Phishing/Suspected Domain",
    "Identified Phishing/Suspected Domain Name"
]
MAX_CONCURRENT = 40
REQUEST_TIMEOUT = 15
DNS_TIMEOUT = 10
FOLLOW_REDIRECTS = True
VERIFY_SSL = True
SUMMARY_CSV = "checker_summary.csv"

# Helper to normalise extracted candidate
def normalize_value(v):
    if pd.isna(v):
        return None
    s = str(v).strip()
    return s if s else None

# DNS resolve (uses aiodns)
async def resolve_domain(resolver, domain):
    try:
        ans = await resolver.gethostbyname(domain, socket.AF_INET)
        if ans and ans.addresses:
            return True, ans.addresses
        return False, []
    except Exception:
        return False, []

# Try HTTP(S)
async def try_http(session, candidate):
    tried = []
    to_try = []
    parsed = urlparse(candidate)
    if parsed.scheme:
        to_try.append(candidate)
    else:
        to_try.extend([candidate, "https://" + candidate, "http://" + candidate])

    for url in to_try:
        if url in tried:
            continue
        tried.append(url)
        try:
            timeout = aiohttp.ClientTimeout(total=REQUEST_TIMEOUT)
            async with session.head(url, allow_redirects=FOLLOW_REDIRECTS, timeout=timeout) as resp:
                if 200 <= resp.status < 400:
                    return True, {"final_url": str(resp.url), "status": resp.status}
        except Exception:
            try:
                async with session.get(url, allow_redirects=FOLLOW_REDIRECTS, timeout=timeout) as resp2:
                    if 200 <= resp2.status < 400:
                        return True, {"final_url": str(resp2.url), "status": resp2.status}
            except Exception:
                continue
    return False, {"error": "all attempts failed"}

# Main checker
async def check_candidate(session, resolver, raw_value, semaphore):
    out = {"raw_value": raw_value}
    val = normalize_value(raw_value)
    out['normalized'] = val
    if not val:
        out.update({'dns_resolved': False, 'http_active': False, 'http_details': None, 'active': False})
        return out

    parsed = urlparse(val if "://" in val else ("http://" + val))
    domain = parsed.hostname
    dns_ok = False
    dns_addrs = []
    if domain:
        try:
            dns_ok, dns_addrs = await resolve_domain(resolver, domain)
        except Exception:
            dns_ok = False

    async with semaphore:
        try:
            http_ok, http_details = await try_http(session, val)
        except Exception as e:
            http_ok, http_details = False, {"error": str(e)}

    out.update({
        'dns_resolved': dns_ok,
        'dns_addresses': dns_addrs,
        'http_active': http_ok,
        'http_details': http_details,
        'active': http_ok  # HTTP success defines activity
    })
    return out

async def process_file(path: Path, summary_rows):
    print(f"Processing: {path}")
    try:
        df_all = pd.read_excel(path, sheet_name=0, engine="openpyxl")
    except Exception as e:
        print(f" Error reading {path.name}: {e}")
        return

    # find domain column
    domain_col = None
    for cname in POSSIBLE_COLUMN_NAMES:
        if cname in df_all.columns:
            domain_col = cname
            break
    if not domain_col:
        for col in df_all.columns:
            sample = df_all[col].dropna().astype(str).head(50).tolist()
            if any("." in s for s in sample):
                domain_col = col
                break
    if not domain_col:
        print(f" No domain column found in {path.name}, skipping.")
        return
    print(f"  Using column: {domain_col}")

    conn = aiohttp.TCPConnector(ssl=ssl.create_default_context() if VERIFY_SSL else False, limit_per_host=MAX_CONCURRENT)
    resolver = aiodns.DNSResolver(timeout=DNS_TIMEOUT)
    timeout = aiohttp.ClientTimeout(total=REQUEST_TIMEOUT)
    semaphore = asyncio.Semaphore(MAX_CONCURRENT)

    async with aiohttp.ClientSession(connector=conn, timeout=timeout) as session:
        tasks = [check_candidate(session, resolver, row[domain_col], semaphore) for _, row in df_all.iterrows()]
        results = []
        for coro in tqdm_asyncio.as_completed(tasks, total=len(tasks), desc=f"  Checking {path.name}"):
            res = await coro
            results.append(res)

    active_records = []
    for idx, (i, row) in enumerate(df_all.iterrows()):
        res = results[idx]
        if res.get('active', False):
            rec = row.to_dict()
            rec.update({
                "_normalized": res.get("normalized"),
                "_dns": res.get("dns_resolved"),
                "_http": res.get("http_active"),
                "_details": str(res.get("http_details")),
            })
            active_records.append(rec)

        summary_rows.append({
            "filename": path.name,
            "row_index": idx,
            "raw_value": res.get("raw_value"),
            "normalized": res.get("normalized"),
            "dns_resolved": res.get("dns_resolved"),
            "http_active": res.get("http_active"),
            "active": res.get("active")
        })

    if active_records:
        out_df = pd.DataFrame(active_records)
        with pd.ExcelWriter(path, engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
            out_df.to_excel(writer, sheet_name=SHEET_NAME_OUT, index=False)
        print(f" Wrote {len(active_records)} active rows to sheet '{SHEET_NAME_OUT}' in {path.name}")
    else:
        print(f" No active rows found in {path.name}")

# ---------------- MAIN FUNCTION ----------------
async def main():
    folder = Path(FOLDER)
    files = list(folder.glob(EXCEL_GLOB))
    if not files:
        print(f"No Excel files found in {folder}")
        return

    summary_rows = []
    for f in files:
        try:
            await process_file(f, summary_rows)
        except Exception as e:
            print(f"Error processing {f.name}: {e}")

    if not summary_rows:
        print(" No data collected.")
        return

    # Save summary
    keys = ["filename","row_index","raw_value","normalized","dns_resolved","http_active","active"]
    df_summary = pd.DataFrame(summary_rows)
    df_summary.to_csv(SUMMARY_CSV, index=False)
    print(f" Summary written to {SUMMARY_CSV}")

    # Filter only active domains
    active_df = df_summary[df_summary["active"] == True]
    if active_df.empty:
        print(" No active domains found.")
        return

    # Save overall active list
    active_file = "ActiveDomains_Final.xlsx"
    active_df.to_excel(active_file, index=False)
    print(f" Saved {len(active_df)} active domains to {active_file}")

    # ---------------- CATEGORIZATION PART ----------------
    label_col = "Phishing/Suspected Domains (i.e. Class Label)"
    phishing_all = []
    suspected_all = []
    others_all = []

    # Iterate again through all Excel files to fetch label column
    for f in files:
        try:
            df = pd.read_excel(f, engine="openpyxl")
            if label_col not in df.columns:
                continue

            domain_col = None
            for cname in POSSIBLE_COLUMN_NAMES:
                if cname in df.columns:
                    domain_col = cname
                    break
            if not domain_col:
                continue

            merged = df.merge(active_df, left_on=domain_col, right_on="normalized", how="inner")
            if merged.empty:
                continue

            merged["_label_norm"] = merged[label_col].astype(str).str.lower().str.strip()

            phishing_part = merged[merged["_label_norm"].str.contains("phish", na=False)]
            suspected_part = merged[merged["_label_norm"].str.contains("suspect|suspicious", na=False)]
            other_part = merged[~(
                merged.index.isin(phishing_part.index) | merged.index.isin(suspected_part.index)
            )]

            phishing_all.append(phishing_part)
            suspected_all.append(suspected_part)
            others_all.append(other_part)

        except Exception as e:
            print(f"⚠️ Error processing labels in {f.name}: {e}")

    phishing_df = pd.concat(phishing_all, ignore_index=True) if phishing_all else pd.DataFrame()
    suspected_df = pd.concat(suspected_all, ignore_index=True) if suspected_all else pd.DataFrame()
    others_df = pd.concat(others_all, ignore_index=True) if others_all else pd.DataFrame()

    phishing_df.to_csv("Active_Phishing.csv", index=False)
    suspected_df.to_csv("Active_Suspected.csv", index=False)
    others_df.to_csv("Active_Others.csv", index=False)

    with pd.ExcelWriter("ActiveDomains_ByCategory.xlsx", engine="openpyxl") as writer:
        if not phishing_df.empty:
            phishing_df.to_excel(writer, sheet_name="Phishing", index=False)
        if not suspected_df.empty:
            suspected_df.to_excel(writer, sheet_name="Suspected", index=False)
        if not others_df.empty:
            others_df.to_excel(writer, sheet_name="Others", index=False)

    print(" Categorized outputs saved:")
    print(f"   • Active_Phishing.csv ({len(phishing_df)})")
    print(f"   • Active_Suspected.csv ({len(suspected_df)})")
    print(f"   • Active_Others.csv ({len(others_df)})")
    print(f"   • ActiveDomains_ByCategory.xlsx created successfully")

# Run main
await main()


Processing: C:\Users\HP\Desktop\PhishingDetection\models\model_training\PhishingUrlData\Mock_Data_01_08_2025.xlsx
  Using column: Identified Phishing/Suspected Domain Name


  Checking Mock_Data_01_08_2025.xlsx: 100%|██████████| 1244/1244 [02:18<00:00,  8.99it/s]


Error processing Mock_Data_01_08_2025.xlsx: [Errno 13] Permission denied: 'C:\\Users\\HP\\Desktop\\PhishingDetection\\models\\model_training\\PhishingUrlData\\Mock_Data_01_08_2025.xlsx'
Processing: C:\Users\HP\Desktop\PhishingDetection\models\model_training\PhishingUrlData\Mock_Data_03_09_2025.xlsx
  Using column: Identified Phishing/Suspected Domain Name


  Checking Mock_Data_03_09_2025.xlsx: 100%|██████████| 1372/1372 [01:02<00:00, 22.05it/s]


✅ Wrote 1327 active rows to sheet 'Active_Domains' in Mock_Data_03_09_2025.xlsx
Processing: C:\Users\HP\Desktop\PhishingDetection\models\model_training\PhishingUrlData\Mock_Data_04_08_2025.xlsx
  Using column: Identified Phishing/Suspected Domain Name


...

⚠️ No active rows found in Mock_Data_28_08_2025.xlsx
Processing: C:\Users\HP\Desktop\PhishingDetection\models\model_training\PhishingUrlData\Mock_Data_29_08_2025.xlsx
  Using column: Identified Phishing/Suspected Domain Name


  Checking Mock_Data_29_08_2025.xlsx: 100%|██████████| 12/12 [00:05<00:00,  2.23it/s]


⚠️ No active rows found in Mock_Data_29_08_2025.xlsx
Processing: C:\Users\HP\Desktop\PhishingDetection\models\model_training\PhishingUrlData\Mock_Data_31_07_2025.xlsx
  Using column: Identified Phishing/Suspected Domain Name


  Checking Mock_Data_31_07_2025.xlsx: 100%|██████████| 302/302 [00:44<00:00,  6.83it/s]


✅ Wrote 20 active rows to sheet 'Active_Domains' in Mock_Data_31_07_2025.xlsx
Processing: C:\Users\HP\Desktop\PhishingDetection\models\model_training\PhishingUrlData\~$Mock_Data_01_08_2025.xlsx
❌ Error reading ~$Mock_Data_01_08_2025.xlsx: [Errno 2] No such file or directory: 'C:\\Users\\HP\\Desktop\\PhishingDetection\\models\\model_training\\PhishingUrlData\\~$Mock_Data_01_08_2025.xlsx'
📄 Summary written to checker_summary.csv
✅ Saved 1640 active domains to ActiveDomains_Final.xlsx
⚠️ Error processing labels in ~$Mock_Data_01_08_2025.xlsx: [Errno 2] No such file or directory: 'C:\\Users\\HP\\Desktop\\PhishingDetection\\models\\model_training\\PhishingUrlData\\~$Mock_Data_01_08_2025.xlsx'
✅ Categorized outputs saved:
   • Active_Phishing.csv (1500)
   • Active_Suspected.csv (8)
   • Active_Others.csv (0)
   • ActiveDomains_ByCategory.xlsx created successfully


In [3]:
import asyncio
import aiohttp
import aiodns
import pandas as pd
import socket
from pathlib import Path
from urllib.parse import urlparse
from tqdm import tqdm
import csv
import ssl
import re
import nest_asyncio

nest_asyncio.apply()

# ===== CONFIG =====
FOLDER = Path("C:\\Users\\HP\\Desktop\\PhishingDetection\\models\\model_training\\PhishingUrlData")
EXCEL_GLOB = "*.xls*"
SHEET_NAME_OUT = "Active_Domains"
POSSIBLE_COLUMN_NAMES = [
    "Identified Phishing/Suspected Domain Name",
    "Identified Phishing/Suspected Domain",
    "Domain",
    "URL"
]
MAX_CONCURRENT = 40
REQUEST_TIMEOUT = 10
DNS_TIMEOUT = 5
FOLLOW_REDIRECTS = True
VERIFY_SSL = True
SUMMARY_CSV = "checker_summary.csv"
LABEL_COL = "Phishing/Suspected Domains (i.e. Class Label)"
MAX_EXCEL_ROWS = 1_000_000
# ==================

def canonical_host(value):
    """Normalize URL/domain into canonical hostname."""
    if pd.isna(value) or not str(value).strip():
        return None
    s = str(value).strip()
    if "://" not in s:
        s = "http://" + s
    try:
        host = urlparse(s).hostname
        if host:
            host = host.lower().rstrip(".")
            if host.startswith("www."):
                host = host[4:]
        return host
    except Exception:
        return None

async def resolve_domain(resolver, domain):
    try:
        ans = await resolver.gethostbyname(domain, socket.AF_INET)
        return bool(ans.addresses), ans.addresses
    except Exception:
        return False, []

async def try_http(session, url):
    """Try HTTP(S) requests and return status."""
    candidates = []
    p = urlparse(url)
    if p.scheme:
        candidates.append(url)
    else:
        candidates += [f"http://{url}", f"https://{url}"]

    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}
    for candidate in candidates:
        try:
            async with session.head(candidate, allow_redirects=FOLLOW_REDIRECTS, timeout=REQUEST_TIMEOUT, headers=headers) as r:
                if 200 <= r.status < 400:
                    return True, r.status
        except Exception:
            try:
                async with session.get(candidate, allow_redirects=FOLLOW_REDIRECTS, timeout=REQUEST_TIMEOUT, headers=headers) as r2:
                    if 200 <= r2.status < 400:
                        return True, r2.status
            except Exception:
                continue
    return False, None

async def check_domain(session, resolver, domain):
    """Perform DNS + HTTP checks for one domain."""
    result = {"raw_value": domain, "host": canonical_host(domain), "active": False}
    host = result["host"]
    if not host:
        return result

    try:
        dns_ok, _ = await resolve_domain(resolver, host)
        http_ok, status = await try_http(session, host)
        result.update({
            "dns_resolved": dns_ok,
            "http_active": http_ok,
            "http_status": status,
            "active": http_ok or dns_ok
        })
    except Exception as e:
        result.update({"error": str(e), "dns_resolved": False, "http_active": False})
    return result

async def process_file(file, summary_rows):
    """Process a single Excel file and append active results."""
    print(f"\n📄 Processing file: {file.name}")
    try:
        df = pd.read_excel(file)
    except Exception as e:
        print(f"❌ Failed to read {file}: {e}")
        return

    domain_col = next((c for c in POSSIBLE_COLUMN_NAMES if c in df.columns), None)
    if not domain_col:
        print(f"⚠️ No domain column found in {file.name}, skipping.")
        return

    domains = df[domain_col].dropna().astype(str).tolist()
    if not domains:
        print(f"⚠️ No domains found in {file.name}.")
        return

    conn = aiohttp.TCPConnector(ssl=ssl.create_default_context() if VERIFY_SSL else False, limit_per_host=MAX_CONCURRENT)
    resolver = aiodns.DNSResolver(timeout=DNS_TIMEOUT)
    semaphore = asyncio.Semaphore(MAX_CONCURRENT)

    async with aiohttp.ClientSession(connector=conn) as session:
        async def sem_task(d):
            async with semaphore:
                return await check_domain(session, resolver, d)

        tasks = [sem_task(d) for d in domains]
        results = []
        for fut in tqdm(asyncio.as_completed(tasks), total=len(tasks), desc=f"  Checking {file.name}"):
            try:
                res = await fut
                results.append(res)
            except Exception as e:
                results.append({"raw_value": None, "error": str(e), "active": False})

    df_results = pd.DataFrame(results)
    df_results["filename"] = file.name
    summary_rows.append(df_results)

    active_rows = df_results[df_results["active"] == True]
    if active_rows.empty:
        print(f"⚠️ No active rows found in {file.name}")
    else:
        try:
            with pd.ExcelWriter(file, engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
                active_rows.to_excel(writer, sheet_name=SHEET_NAME_OUT, index=False)
            print(f"✅ {len(active_rows)} active URLs written to '{SHEET_NAME_OUT}' in {file.name}")
        except Exception as e:
            fallback = file.with_name(file.stem + "_Active.xlsx")
            active_rows.to_excel(fallback, index=False)
            print(f"⚠️ Could not append to {file.name}, saved to {fallback.name}. Error: {e}")

async def main():
    folder = Path(FOLDER)
    files = list(folder.glob(EXCEL_GLOB))
    if not files:
        print("❌ No Excel files found.")
        return

    summary_rows = []
    for f in files:
        await process_file(f, summary_rows)

    if not summary_rows:
        print("⚠️ No data processed.")
        return

    df_summary = pd.concat(summary_rows, ignore_index=True)
    df_summary.to_csv(SUMMARY_CSV, index=False)
    print(f"\n📊 Summary saved to {SUMMARY_CSV}")

    active_df = df_summary[df_summary["active"] == True].copy()
    if active_df.empty:
        print("⚠️ No active domains overall.")
        return

    active_df.to_excel("ActiveDomains_Final.xlsx", index=False)
    print(f"✅ ActiveDomains_Final.xlsx created with {len(active_df)} entries.")

    # ---- Categorization (deduplicated) ----
    phishing, suspected, others = [], [], []

    # ✅ Deduplicate active hosts to prevent merge explosion
    active_hosts_df = active_df[['host']].drop_duplicates()

    for f in files:
        try:
            df = pd.read_excel(f)
        except Exception:
            continue

        if LABEL_COL not in df.columns:
            continue

        domain_col = next((c for c in POSSIBLE_COLUMN_NAMES if c in df.columns), None)
        if not domain_col:
            continue

        df["_host"] = df[domain_col].apply(canonical_host)
        merged = pd.merge(df, active_hosts_df, left_on="_host", right_on="host", how="inner")

        if merged.empty:
            continue

        merged["_label_norm"] = merged[LABEL_COL].astype(str).str.lower().str.strip()

        phishing.append(merged[merged["_label_norm"].str.contains("phish", na=False)])
        suspected.append(merged[merged["_label_norm"].str.contains("suspect|suspicious", na=False)])
        others.append(merged[~merged["_label_norm"].str.contains("phish|suspect|suspicious", na=False)])

    phishing_df = pd.concat(phishing, ignore_index=True) if phishing else pd.DataFrame()
    suspected_df = pd.concat(suspected, ignore_index=True) if suspected else pd.DataFrame()
    others_df = pd.concat(others, ignore_index=True) if others else pd.DataFrame()

    phishing_df.to_csv("Active_Phishing.csv", index=False)
    suspected_df.to_csv("Active_Suspected.csv", index=False)
    others_df.to_csv("Active_Others.csv", index=False)

    print("\n🧾 CSVs saved:")
    print(f"   • Active_Phishing.csv ({len(phishing_df)})")
    print(f"   • Active_Suspected.csv ({len(suspected_df)})")
    print(f"   • Active_Others.csv ({len(others_df)})")

    # ---- Excel Export with Auto-Split ----
    def write_large_df_to_excel(writer, df, base_name):
        if df.empty:
            return
        total = len(df)
        if total <= MAX_EXCEL_ROWS:
            df.to_excel(writer, sheet_name=base_name, index=False)
            print(f"  ✅ Sheet '{base_name}' written with {total} rows")
        else:
            parts = (total // MAX_EXCEL_ROWS) + 1
            for i in range(parts):
                start = i * MAX_EXCEL_ROWS
                end = min((i + 1) * MAX_EXCEL_ROWS, total)
                chunk = df.iloc[start:end]
                chunk.to_excel(writer, sheet_name=f"{base_name}_{i+1}", index=False)
                print(f"  ➕ Wrote chunk {i+1}/{parts} ({len(chunk)} rows) to '{base_name}_{i+1}'")

    with pd.ExcelWriter("ActiveDomains_ByCategory.xlsx", engine="openpyxl") as w:
        write_large_df_to_excel(w, phishing_df, "Phishing")
        write_large_df_to_excel(w, suspected_df, "Suspected")
        write_large_df_to_excel(w, others_df, "Others")

    # ---- Summary Print ----
    print("\n📈 Summary Report:")
    print(f"   Phishing:  {len(phishing_df)} rows")
    print(f"   Suspected: {len(suspected_df)} rows")
    print(f"   Others:    {len(others_df)} rows")
    print(f"   Total Active: {len(active_df)}")
    print("\n✅ Categorized Excel created: ActiveDomains_ByCategory.xlsx (auto-split if >1M rows)")

# ---- Run ----
await main()



📄 Processing file: Mock_Data_01_08_2025.xlsx


  Checking Mock_Data_01_08_2025.xlsx: 100%|██████████| 1244/1244 [02:01<00:00, 10.26it/s]


✅ 687 active URLs written to 'Active_Domains' in Mock_Data_01_08_2025.xlsx

📄 Processing file: Mock_Data_03_09_2025.xlsx


  Checking Mock_Data_03_09_2025.xlsx: 100%|██████████| 1372/1372 [00:41<00:00, 33.07it/s]


⚠️ Could not append to Mock_Data_03_09_2025.xlsx, saved to Mock_Data_03_09_2025_Active.xlsx. Error: [Errno 13] Permission denied: 'C:\\Users\\HP\\Desktop\\PhishingDetection\\models\\model_training\\PhishingUrlData\\Mock_Data_03_09_2025.xlsx'

📄 Processing file: Mock_Data_03_09_2025_Active.xlsx
⚠️ No domain column found in Mock_Data_03_09_2025_Active.xlsx, skipping.

📄 Processing file: Mock_Data_04_08_2025.xlsx


...

✅ 1 active URLs written to 'Active_Domains' in Mock_Data_28_08_2025.xlsx

📄 Processing file: Mock_Data_29_08_2025.xlsx


  Checking Mock_Data_29_08_2025.xlsx: 100%|██████████| 12/12 [00:00<00:00, 12.77it/s]


✅ 2 active URLs written to 'Active_Domains' in Mock_Data_29_08_2025.xlsx

📄 Processing file: Mock_Data_31_07_2025.xlsx


  Checking Mock_Data_31_07_2025.xlsx: 100%|██████████| 302/302 [00:45<00:00,  6.58it/s]
C:\Users\HP\AppData\Local\Temp\ipykernel_22532\2879773346.py:173: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_summary = pd.concat(summary_rows, ignore_index=True)


✅ 244 active URLs written to 'Active_Domains' in Mock_Data_31_07_2025.xlsx

📄 Processing file: ~$Mock_Data_03_09_2025.xlsx
❌ Failed to read C:\Users\HP\Desktop\PhishingDetection\models\model_training\PhishingUrlData\~$Mock_Data_03_09_2025.xlsx: [Errno 13] Permission denied: 'C:\\Users\\HP\\Desktop\\PhishingDetection\\models\\model_training\\PhishingUrlData\\~$Mock_Data_03_09_2025.xlsx'

📊 Summary saved to checker_summary.csv
✅ ActiveDomains_Final.xlsx created with 2501 entries.

🧾 CSVs saved:
   • Active_Phishing.csv (2465)
   • Active_Suspected.csv (36)
   • Active_Others.csv (0)
  ✅ Sheet 'Phishing' written with 2465 rows
  ✅ Sheet 'Suspected' written with 36 rows

📈 Summary Report:
   Phishing:  2465 rows
   Suspected: 36 rows
   Others:    0 rows
   Total Active: 2501

✅ Categorized Excel created: ActiveDomains_ByCategory.xlsx (auto-split if >1M rows)


In [8]:
import asyncio
import aiohttp
import aiodns
import pandas as pd
import socket
from pathlib import Path
from urllib.parse import urlparse
import csv
import ssl
import re
import nest_asyncio
from tqdm import tqdm
from datetime import datetime
import hashlib
from bs4 import BeautifulSoup

nest_asyncio.apply()

# ===== CONFIG =====
FOLDER = Path("C:\\Users\\HP\\Desktop\\PhishingDetection\\models\\model_training\\PhishingUrlData")
EXCEL_GLOB = "*.xls*"
SHEET_NAME_OUT = "Active_Domains"
POSSIBLE_COLUMN_NAMES = [
    "Identified Phishing/Suspected Domain Name",
    "Identified Phishing/Suspected Domain",
    "Domain",
    "URL"
]
MAX_CONCURRENT = 40
REQUEST_TIMEOUT = 10
DNS_TIMEOUT = 5
FOLLOW_REDIRECTS = True
VERIFY_SSL = False  # Many phishing sites have invalid certs
SUMMARY_CSV = "checker_summary.csv"
LABEL_COL = "Phishing/Suspected Domains (i.e. Class Label)"
MAX_EXCEL_ROWS = 1_000_000

# ===== PHISHING DETECTION CONFIG =====
SUSPICIOUS_KEYWORDS = [
    'login', 'signin', 'verify', 'account', 'secure', 'update', 
    'suspended', 'locked', 'confirm', 'banking', 'paypal', 'amazon',
    'password', 'credential', 'validation', 'authentication'
]
BRAND_NAMES = [
    'paypal', 'amazon', 'microsoft', 'apple', 'google', 'facebook',
    'instagram', 'twitter', 'netflix', 'bank', 'chase', 'wellsfargo',
    'citibank', 'dhl', 'fedex', 'usps', 'irs', 'irs.gov'
]
# ==================

def canonical_host(value):
    """Normalize URL/domain into canonical hostname."""
    if pd.isna(value) or not str(value).strip():
        return None
    s = str(value).strip()
    if "://" not in s:
        s = "http://" + s
    try:
        host = urlparse(s).hostname
        if host:
            host = host.lower().rstrip(".")
            if host.startswith("www."):
                host = host[4:]
        return host
    except Exception:
        return None

def calculate_phishing_score(url, html_content, status_code, ssl_valid):
    """Calculate phishing likelihood score based on multiple indicators."""
    score = 0
    indicators = []
    
    url_lower = url.lower()
    
    # 1. Check for suspicious URL patterns
    if re.search(r'\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}', url):  # IP address
        score += 25
        indicators.append("IP_ADDRESS_IN_URL")
    
    if url.count('.') > 4:  # Too many subdomains
        score += 15
        indicators.append("EXCESSIVE_SUBDOMAINS")
    
    if url.count('-') > 3:  # Excessive hyphens
        score += 10
        indicators.append("EXCESSIVE_HYPHENS")
    
    if '@' in url:  # @ symbol in URL
        score += 20
        indicators.append("AT_SYMBOL_IN_URL")
    
    # 2. Check for suspicious keywords
    keyword_matches = [kw for kw in SUSPICIOUS_KEYWORDS if kw in url_lower]
    if keyword_matches:
        score += len(keyword_matches) * 5
        indicators.append(f"SUSPICIOUS_KEYWORDS:{','.join(keyword_matches[:3])}")
    
    # 3. Check for brand impersonation
    brand_matches = [brand for brand in BRAND_NAMES if brand in url_lower]
    if brand_matches:
        # Check if it's NOT the official domain
        parsed = urlparse(url if '://' in url else 'http://' + url)
        hostname = parsed.hostname or ''
        for brand in brand_matches:
            # If brand name is in subdomain/path but not main domain
            if brand in hostname and not hostname.endswith(f'{brand}.com'):
                score += 20
                indicators.append(f"BRAND_IMPERSONATION:{brand}")
    
    # 4. Check URL length
    if len(url) > 75:
        score += 10
        indicators.append("LONG_URL")
    
    # 5. SSL Certificate issues
    if not ssl_valid:
        score += 15
        indicators.append("INVALID_SSL")
    
    # 6. Analyze HTML content if available
    if html_content:
        soup = BeautifulSoup(html_content, 'html.parser')
        
        # Check for forms (common in phishing)
        forms = soup.find_all('form')
        if forms:
            score += 10
            indicators.append(f"FORMS_FOUND:{len(forms)}")
            
            # Check for password inputs
            password_inputs = soup.find_all('input', {'type': 'password'})
            if password_inputs:
                score += 15
                indicators.append("PASSWORD_INPUT_FOUND")
        
        # Check for hidden iframes (common phishing technique)
        iframes = soup.find_all('iframe', style=re.compile(r'display\s*:\s*none', re.I))
        if iframes:
            score += 20
            indicators.append("HIDDEN_IFRAMES")
        
        # Check for external form actions
        for form in forms:
            action = form.get('action', '')
            if action and action.startswith('http') and urlparse(action).hostname != urlparse(url).hostname:
                score += 20
                indicators.append("EXTERNAL_FORM_ACTION")
                break
        
        # Check page title and content for brand names
        title = soup.find('title')
        if title:
            title_text = title.get_text().lower()
            content_brands = [b for b in BRAND_NAMES if b in title_text]
            if content_brands and not any(b in url_lower for b in content_brands):
                score += 15
                indicators.append(f"BRAND_IN_TITLE_NOT_URL:{content_brands[0]}")
    
    # 7. Check for redirects (status codes)
    if status_code and status_code in [301, 302, 307, 308]:
        score += 10
        indicators.append(f"REDIRECT:{status_code}")
    
    return min(score, 100), indicators

async def resolve_domain(resolver, domain):
    try:
        ans = await resolver.gethostbyname(domain, socket.AF_INET)
        return bool(ans.addresses), ans.addresses
    except Exception:
        return False, []

async def try_http_advanced(session, url):
    """Enhanced HTTP check with content analysis."""
    candidates = []
    p = urlparse(url)
    if p.scheme:
        candidates.append(url)
    else:
        candidates += [f"https://{url}", f"http://{url}"]

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
        "Accept-Language": "en-US,en;q=0.5"
    }
    
    for candidate in candidates:
        try:
            # Try HEAD first
            async with session.head(candidate, allow_redirects=FOLLOW_REDIRECTS, 
                                   timeout=REQUEST_TIMEOUT, headers=headers, ssl=False) as r:
                if 200 <= r.status < 400:
                    # Get full content with GET
                    try:
                        async with session.get(candidate, allow_redirects=FOLLOW_REDIRECTS,
                                             timeout=REQUEST_TIMEOUT, headers=headers, ssl=False) as r2:
                            html = await r2.text()
                            ssl_valid = r2.url.scheme == 'https'
                            return True, r2.status, html, ssl_valid, str(r2.url)
                    except Exception:
                        return True, r.status, None, False, candidate
        except Exception:
            try:
                # Direct GET if HEAD fails
                async with session.get(candidate, allow_redirects=FOLLOW_REDIRECTS,
                                     timeout=REQUEST_TIMEOUT, headers=headers, ssl=False) as r2:
                    if 200 <= r2.status < 400:
                        html = await r2.text()
                        ssl_valid = r2.url.scheme == 'https'
                        return True, r2.status, html, ssl_valid, str(r2.url)
            except Exception:
                continue
    
    return False, None, None, False, None

async def check_domain(session, resolver, domain):
    """Enhanced domain check with phishing detection."""
    result = {
        "raw_value": domain, 
        "host": canonical_host(domain), 
        "active": False,
        "phishing_score": 0,
        "phishing_indicators": "",
        "verified_phishing": False,
        "content_hash": None,
        "final_url": None
    }
    
    host = result["host"]
    if not host:
        return result

    try:
        dns_ok, ip_addresses = await resolve_domain(resolver, host)
        http_ok, status, html, ssl_valid, final_url = await try_http_advanced(session, host)
        
        result.update({
            "dns_resolved": dns_ok,
            "http_active": http_ok,
            "http_status": status,
            "active": http_ok or dns_ok,
            "ip_addresses": ','.join(ip_addresses) if ip_addresses else None,
            "ssl_valid": ssl_valid,
            "final_url": final_url
        })
        
        if http_ok and html:
            # Calculate phishing score
            score, indicators = calculate_phishing_score(
                final_url or host, html, status, ssl_valid
            )
            result["phishing_score"] = score
            result["phishing_indicators"] = '|'.join(indicators)
            result["verified_phishing"] = score >= 50  # Threshold for likely phishing
            result["content_hash"] = hashlib.md5(html.encode()).hexdigest()[:16]
        
    except Exception as e:
        result.update({
            "error": str(e), 
            "dns_resolved": False, 
            "http_active": False
        })
    
    return result

async def process_file(file, summary_rows):
    """Process a single Excel file and append active results."""
    print(f"\n📄 Processing file: {file.name}")
    try:
        df = pd.read_excel(file)
    except Exception as e:
        print(f"❌ Failed to read {file}: {e}")
        return

    domain_col = next((c for c in POSSIBLE_COLUMN_NAMES if c in df.columns), None)
    if not domain_col:
        print(f"⚠️ No domain column found in {file.name}, skipping.")
        return

    domains = df[domain_col].dropna().astype(str).tolist()
    if not domains:
        print(f"⚠️ No domains found in {file.name}.")
        return

    conn = aiohttp.TCPConnector(ssl=False, limit_per_host=MAX_CONCURRENT)
    resolver = aiodns.DNSResolver(timeout=DNS_TIMEOUT)
    semaphore = asyncio.Semaphore(MAX_CONCURRENT)

    async with aiohttp.ClientSession(connector=conn) as session:
        async def sem_task(d):
            async with semaphore:
                return await check_domain(session, resolver, d)

        tasks = [sem_task(d) for d in domains]
        results = []
        for fut in tqdm(asyncio.as_completed(tasks), total=len(tasks), 
                       desc=f"  Checking {file.name}"):
            try:
                res = await fut
                results.append(res)
            except Exception as e:
                results.append({
                    "raw_value": None, 
                    "error": str(e), 
                    "active": False,
                    "phishing_score": 0
                })

    df_results = pd.DataFrame(results)
    df_results["filename"] = file.name
    df_results["check_timestamp"] = datetime.now().isoformat()
    summary_rows.append(df_results)

    active_rows = df_results[df_results["active"] == True]
    if active_rows.empty:
        print(f"⚠️ No active rows found in {file.name}")
    else:
        try:
            with pd.ExcelWriter(file, engine="openpyxl", mode="a", 
                              if_sheet_exists="replace") as writer:
                active_rows.to_excel(writer, sheet_name=SHEET_NAME_OUT, index=False)
            print(f"✅ {len(active_rows)} active URLs written to '{SHEET_NAME_OUT}' in {file.name}")
            
            # Print phishing stats
            verified = len(active_rows[active_rows["verified_phishing"] == True])
            if verified > 0:
                print(f"   🚨 {verified} domains verified as likely phishing (score ≥50)")
        except Exception as e:
            fallback = file.with_name(file.stem + "_Active.xlsx")
            active_rows.to_excel(fallback, index=False)
            print(f"⚠️ Could not append to {file.name}, saved to {fallback.name}. Error: {e}")

async def main():
    folder = Path(FOLDER)
    files = list(folder.glob(EXCEL_GLOB))
    if not files:
        print("❌ No Excel files found.")
        return

    summary_rows = []
    for f in files:
        await process_file(f, summary_rows)

    if not summary_rows:
        print("⚠️ No data processed.")
        return

    df_summary = pd.concat(summary_rows, ignore_index=True)
    df_summary.to_csv(SUMMARY_CSV, index=False)
    print(f"\n📊 Summary saved to {SUMMARY_CSV}")

    active_df = df_summary[df_summary["active"] == True].copy()
    if active_df.empty:
        print("⚠️ No active domains overall.")
        return

    active_df.to_excel("ActiveDomains_Final.xlsx", index=False)
    print(f"✅ ActiveDomains_Final.xlsx created with {len(active_df)} entries.")

    # ---- Enhanced Categorization with Phishing Verification ----
    phishing, suspected, others, verified_phishing = [], [], [], []

    active_hosts_df = active_df[['host']].drop_duplicates()

    for f in files:
        try:
            df = pd.read_excel(f)
        except Exception:
            continue

        if LABEL_COL not in df.columns:
            continue

        domain_col = next((c for c in POSSIBLE_COLUMN_NAMES if c in df.columns), None)
        if not domain_col:
            continue

        df["_host"] = df[domain_col].apply(canonical_host)
        merged = pd.merge(df, active_df, left_on="_host", right_on="host", how="inner")

        if merged.empty:
            continue

        merged["_label_norm"] = merged[LABEL_COL].astype(str).str.lower().str.strip()

        phishing.append(merged[merged["_label_norm"].str.contains("phish", na=False)])
        suspected.append(merged[merged["_label_norm"].str.contains("suspect|suspicious", na=False)])
        others.append(merged[~merged["_label_norm"].str.contains("phish|suspect|suspicious", na=False)])

    phishing_df = pd.concat(phishing, ignore_index=True) if phishing else pd.DataFrame()
    suspected_df = pd.concat(suspected, ignore_index=True) if suspected else pd.DataFrame()
    others_df = pd.concat(others, ignore_index=True) if others else pd.DataFrame()

    # ---- NEW: Verified Phishing (high-confidence based on analysis) ----
    verified_phishing_df = active_df[active_df["verified_phishing"] == True].copy()

    phishing_df.to_csv("Active_Phishing.csv", index=False)
    suspected_df.to_csv("Active_Suspected.csv", index=False)
    others_df.to_csv("Active_Others.csv", index=False)
    verified_phishing_df.to_csv("Verified_Phishing_HighConfidence.csv", index=False)

    print("\n🧾 CSVs saved:")
    print(f"   • Active_Phishing.csv ({len(phishing_df)})")
    print(f"   • Active_Suspected.csv ({len(suspected_df)})")
    print(f"   • Active_Others.csv ({len(others_df)})")
    print(f"   🚨 Verified_Phishing_HighConfidence.csv ({len(verified_phishing_df)}) - Score ≥50")

    # ---- Phishing Score Analysis ----
    if not active_df.empty:
        print("\n📈 Phishing Score Distribution:")
        print(f"   High Risk (≥70):  {len(active_df[active_df['phishing_score'] >= 70])}")
        print(f"   Medium Risk (50-69): {len(active_df[(active_df['phishing_score'] >= 50) & (active_df['phishing_score'] < 70)])}")
        print(f"   Low Risk (30-49): {len(active_df[(active_df['phishing_score'] >= 30) & (active_df['phishing_score'] < 50)])}")
        print(f"   Minimal Risk (<30): {len(active_df[active_df['phishing_score'] < 30])}")

    # ---- Excel Export with Auto-Split ----
    def write_large_df_to_excel(writer, df, base_name):
        if df.empty:
            return
        total = len(df)
        if total <= MAX_EXCEL_ROWS:
            df.to_excel(writer, sheet_name=base_name, index=False)
            print(f"  ✅ Sheet '{base_name}' written with {total} rows")
        else:
            parts = (total // MAX_EXCEL_ROWS) + 1
            for i in range(parts):
                start = i * MAX_EXCEL_ROWS
                end = min((i + 1) * MAX_EXCEL_ROWS, total)
                chunk = df.iloc[start:end]
                chunk.to_excel(writer, sheet_name=f"{base_name}_{i+1}", index=False)
                print(f"  ➕ Wrote chunk {i+1}/{parts} ({len(chunk)} rows) to '{base_name}_{i+1}'")

    with pd.ExcelWriter("ActiveDomains_ByCategory.xlsx", engine="openpyxl") as w:
        write_large_df_to_excel(w, phishing_df, "Phishing")
        write_large_df_to_excel(w, suspected_df, "Suspected")
        write_large_df_to_excel(w, others_df, "Others")
        write_large_df_to_excel(w, verified_phishing_df, "Verified_HighConf")

    # ---- Summary Print ----
    print("\n📈 Summary Report:")
    print(f"   Phishing:  {len(phishing_df)} rows")
    print(f"   Suspected: {len(suspected_df)} rows")
    print(f"   Others:    {len(others_df)} rows")
    print(f"   🚨 Verified High Confidence: {len(verified_phishing_df)} rows")
    print(f"   Total Active: {len(active_df)}")
    print("\n✅ Categorized Excel created: ActiveDomains_ByCategory.xlsx")
    print("✅ Use 'Verified_Phishing_HighConfidence.csv' for model training!")

# ---- Run ----
await main()


📄 Processing file: Mock_Data_01_08_2025.xlsx


  Checking Mock_Data_01_08_2025.xlsx: 100%|██████████| 1244/1244 [02:19<00:00,  8.92it/s]


✅ 686 active URLs written to 'Active_Domains' in Mock_Data_01_08_2025.xlsx
   🚨 13 domains verified as likely phishing (score ≥50)

📄 Processing file: Mock_Data_03_09_2025.xlsx


  Checking Mock_Data_03_09_2025.xlsx: 100%|██████████| 1372/1372 [01:29<00:00, 15.41it/s]


✅ 1367 active URLs written to 'Active_Domains' in Mock_Data_03_09_2025.xlsx
   🚨 2 domains verified as likely phishing (score ≥50)

📄 Processing file: Mock_Data_03_09_2025_Active.xlsx
⚠️ No domain column found in Mock_Data_03_09_2025_Active.xlsx, skipping.

📄 Processing file: Mock_Data_04_08_2025.xlsx


...

✅ 2 active URLs written to 'Active_Domains' in Mock_Data_29_08_2025.xlsx

📄 Processing file: Mock_Data_31_07_2025.xlsx


  Checking Mock_Data_31_07_2025.xlsx: 100%|██████████| 302/302 [00:45<00:00,  6.64it/s]


✅ 244 active URLs written to 'Active_Domains' in Mock_Data_31_07_2025.xlsx

📊 Summary saved to checker_summary.csv


C:\Users\HP\AppData\Local\Temp\ipykernel_22532\989373228.py:349: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_summary = pd.concat(summary_rows, ignore_index=True)


✅ ActiveDomains_Final.xlsx created with 2500 entries.

🧾 CSVs saved:
   • Active_Phishing.csv (1546904)
   • Active_Suspected.csv (36)
   • Active_Others.csv (0)
   🚨 Verified_Phishing_HighConfidence.csv (16) - Score ≥50

📈 Phishing Score Distribution:
   High Risk (≥70):  1
   Medium Risk (50-69): 15
   Low Risk (30-49): 42
   Minimal Risk (<30): 2442
  ➕ Wrote chunk 1/2 (1000000 rows) to 'Phishing_1'
  ➕ Wrote chunk 2/2 (546904 rows) to 'Phishing_2'
  ✅ Sheet 'Suspected' written with 36 rows
  ✅ Sheet 'Verified_HighConf' written with 16 rows

📈 Summary Report:
   Phishing:  1546904 rows
   Suspected: 36 rows
   Others:    0 rows
   🚨 Verified High Confidence: 16 rows
   Total Active: 2500

✅ Categorized Excel created: ActiveDomains_ByCategory.xlsx
✅ Use 'Verified_Phishing_HighConfidence.csv' for model training!


In [10]:
import pandas

df = pd.read_csv("C:\\Users\\HP\\Desktop\\PhishingDetection\\models\\preprocessing\\codes\\Active_Phishing.csv")
df.head()
print(len(df))

1546904


In [9]:
import pandas

df = pd.read_csv("C:\\Users\\HP\\Desktop\\PhishingDetection\\models\\preprocessing\\codes\\Active_Suspected.csv")
df.head()
print(len(df))

36


In [6]:
df = pd.read_excel("C:\\Users\\HP\\Desktop\\PhishingDetection\\models\\preprocessing\\codes\\ActiveDomains_final.xlsx")
df.head()
print(len(df))

2501


In [7]:
import pandas

df = pd.read_csv("C:\\Users\\HP\\Desktop\\PhishingDetection\\models\\preprocessing\\codes\\checker_summary.csv")
df.head()
print(len(df))

3287
